In [1]:
import numpy as np
import random
import requests
import geopy
import pprint as pp
from typing import List, Tuple
from scipy.stats import beta

In [202]:
API_URL = "http://routing.openstreetmap.de/routed-foot/route/v1/{profile}/{lon1},{lat1};{lon2},{lat2}"
GEOLOCATOR = geopy.geocoders.Nominatim(user_agent="My Geocoder")

class Location:
    def __init__(self, name: str, lat: float = None, lon: float = None):
        self.name = name
        self.lat = lat
        self.lon = lon

    def __repr__(self):
        return str(self.__dict__)

    @classmethod
    def from_name(cls, name: str):
        coords = GEOLOCATOR.geocode(name, exactly_one=True)
        if coords:
            return Location(name=name, lat=coords.latitude, lon=coords.longitude)
        else:
            return None

    @classmethod
    def get_routes(cls, profile: str, lat1: float, lon1: float, lat2: float, lon2: float, overview: str = "simplified"):
        """
        Get the routes between two points using the OSRM API.
        - profile: Either "walking" or "driving"
        - lat1, lon1: Coordinates of the first location
        - lat2, lon2: Coordinates of the second location
        Returns the distance in meters.
        """
        url = API_URL.format(profile=profile, lat1=lat1, lon1=lon1, lat2=lat2, lon2=lon2)
        print(url)
        params = {"overview": overview}
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            #distance = data["routes"][0]["distance"]  # distance in meters
            return data["routes"]
        else:
            raise Exception(f"Error: {response.status_code} - {response.text}")
    
    @classmethod
    def get_distance(cls, profile: str, lat1: float, lon1: float, lat2: float, lon2: float):
        """
        Get the distance between two points using the OSRM API.
        - profile: Either "walking" or "driving"
        - lat1, lon1: Coordinates of the first location
        - lat2, lon2: Coordinates of the second location
        Returns the distance in meters.
        """
        return cls.get_routes(profile, lat1, lon1, lat2, lon2, "false")[0]["distance"]

    @staticmethod
    def get_duration_by_distance(distance: float, speed: float):
        """
        Calculate duration by dividing distance by speed.
        - distance: in meters
        - speed: in meters per second
        Returns the duration in minutes.
        """
        duration_seconds = distance / speed
        return duration_seconds / 60.0  # convert to minutes

    def get_walking_duration_to(self, other, speed_mph: float = 3):
        # Get the distance in meters for walking
        distance = Location.get_distance("walking", self.lat, self.lon, other.lat, other.lon)
        # Convert MPH to M/S
        speed_mps = speed_mph * 0.44704
        return self.get_duration_by_distance(distance, speed_mps)

    def get_driving_duration_to(self, other, speed_mph: float = 10):
        # Get the distance in meters for driving
        distance = Location.get_distance("driving", self.lat, self.lon, other.lat, other.lon)
        # Convert MPH to M/S
        speed_mps = speed_mph * 0.44704
        return self.get_duration_by_distance(distance, speed_mps)


In [203]:
class ParkingStructure:
    def __init__(self, index: int, location: Location, is_full: bool = False):
        self.index = index
        self.location = location  # Location object representing the parking structure

    def __repr__(self):
        return str(self.__dict__)

In [204]:
class QLearningParking:
    def __init__(self,
                 parking_structures: List[ParkingStructure],
                 start: Location,
                 end: Location,
                 prior: List[List[int]] = None,
                 alpha: float = 0.1,
                 gamma: float = 0.9,
                 epsilon: float= 0.1,
                 precomputed_duration: List[List[float]] = None):
        self.parking_structures = parking_structures
        self.start = start
        self.end = end
        self.alpha = alpha # Learning rate
        self.gamma = gamma # Discount factor
        self.epsilon = epsilon # Exploration rate

        # Initialize prior belief about the probability of finding parking at each structure using the Beta distribution.
        # prior_alpha represents success counts, prior_beta represents fail counts
        if prior:
            self.prior = prior
        else:
            self.prior = [[1.0, 1.0]] * len(parking_structures) # Uniform prior

        if precomputed_duration:
            self.duration_matrix = precomputed_duration
        else:
            self.duration_matrix = QLearningParking.precompute_duration(self.parking_structures)

        self.start_duration = [self.start.get_driving_duration_to(parking.location) for parking in self.parking_structures]
        self.end_duration = [parking.location.get_walking_duration_to(self.end) for parking in self.parking_structures]

        self.table = dict()


    @classmethod
    def success_prob_to_prior(cls, prob: float):
        return [int(round(prob * 100.0)), int(round((1 - prob) * 100.0))]

    @classmethod
    def precompute_duration(cls, parking_structures: List[ParkingStructure]):
        """
        Precompute the duration of time (driving) between all pairs of parking structures
        and store them in a matrix.
        """
        num_structures = len(parking_structures)
        duration_matrix = np.zeros((num_structures, num_structures))
        print("Precomputing duration of time between all pairs of parking structures.")
        for i in range(num_structures):
            for j in range(i + 1, num_structures):
                # Calculate driving durations between parking structures i and j

                duration = parking_structures[i].location.get_driving_duration_to(parking_structures[j].location)
                print(parking_structures[i].location.name, "->", parking_structures[j].location.name, ":", duration, "mins")
                duration_matrix[i][j] = duration
                duration_matrix[j][i] = duration # Mirror the values for j->i
        return duration_matrix

    def choose_action(self, state: Tuple[int], valid_actions: List[int]):
        """
        Using Epsilon-greedy, choose an action (parking structure).
        - valid_actions: List of indices of parking structures that are not full.
        - state: Current state as a list.
        """

        # Ensure the current state is in the Q-table
        if state not in self.table:
            self.table[state] = np.zeros(len(self.parking_structures))

        if random.uniform(0, 1) < self.epsilon:
            # Choose a random valid action
            action = random.choice(valid_actions)
        else:
            # Choose the action with the highest Q-value among valid actions
            mask = np.ones_like(self.table[state], dtype=bool)
            for visited in state:
                mask[visited] = False

            # mask off indices of parking structures we already visited
            masked_q_values = np.where(mask, self.table[state], -np.inf)
            action = int(np.argmax(masked_q_values))

        return action

    def get_reward(self, current: ParkingStructure, next_: ParkingStructure, found_parking: bool):
        """
        Determine reward
        """
        reward = 0.0
        if current == self.start:
            # driving duration from start to first parking tried
            reward -= self.start_duration[next_.index]
        else:
          if found_parking:
              # success, found parking
              self.prior[current.index][0] += 1
              # walking duration to final destination from successful parking
              reward -= self.end_duration[current.index]
          else:
              # failure, did not find parking
              self.prior[current.index][1] += 1

              # have to spend time driving to the next parking structure
              reward -= self.duration_matrix[current.index][next_.index]

        return reward

    def update(self, state: List[int], action: int, reward: float, next_state: List[int]):
        """
        Update Q-value table
        - state: Current state as a list (binary vector indicating visited parking structures)
        - action: The index of the action (parking structure) taken
        - reward: The reward received
        - next_state: Next state as a list
        """

        # Ensure the current and next states are in the Q-table
        if state not in self.table:
            self.table[state] = np.zeros(len(self.parking_structures))

        if next_state not in self.table:
            self.table[next_state] = np.zeros(len(self.parking_structures))

        # Get the current Q-value
        current_q = self.table[state][action]

        # Calculate the maximum Q-value for the next state
        valid_future_q_indices = list(set(range(len(self.parking_structures))) - set(next_state))

        if valid_future_q_indices:
            max_future_q = max(self.table[next_state][valid_future_q_indices])
        else:
            max_future_q = 0.0

        # Update Q-value using the Bellman equation
        self.table[state][action] = current_q + self.alpha * (reward + self.gamma * max_future_q - current_q)

    def train(self, episodes: int = 1000):
        """
        Train the model
        """
        for episode in range(episodes):
            state = tuple()
            current = self.start
            while True:
                # Determine valid actions based on state
                valid_actions = list(set(range(len(self.parking_structures))) - set(state))
                if not valid_actions:
                    # No valid actions left, didn't find parking, apply large negative reward.
                    reward = -9999
                    break

                action = self.choose_action(state, valid_actions)

                next_state = state + tuple((action,))
                next_ = self.parking_structures[action]
                sample = beta.rvs(*self.prior[action])
                found_parking = sample < random.random()

                reward = self.get_reward(current, next_, found_parking)
                #print(action, self.table[state], reward)
                self.update(state, action, reward, next_state)
                #print("updated", action, self.table[state])
                state = next_state
                current = next_

In [205]:
# Prior for likelihood of finding parking at each parking structure
parking_probability_map = {
    "Roble Field Garage" : 0.80,
    "Via Ortega Garage": 0.50,
    "Stock Farm Garage": 0.50,
    "Tressider Student Union": 0.10,
    "Wilbur Field Garage": 0.50,
}

parking_structures = []
prior = []
for i, (name, prob) in enumerate(parking_probability_map.items()):
    prior.append(QLearningParking.success_prob_to_prior(prob))
    location = Location.from_name(name)
    parking_structures.append(ParkingStructure(i, location))

start_point_name = "Galvez Street & El Camino Real"
start_point = Location.from_name(start_point_name)
end_point_name = "NVIDIA Auditorium"
end_point = Location.from_name(end_point_name)
print(start_point)
print(end_point)
print(prior)

{'name': 'Galvez Street & El Camino Real', 'lat': 37.4369847, 'lon': -122.1611878}
{'name': 'NVIDIA Auditorium', 'lat': 37.4281301, 'lon': -122.1742004}
[[80, 20], [50, 50], [50, 50], [10, 90], [50, 50]]


In [206]:
q_learn = QLearningParking(parking_structures=parking_structures,
                           start=start_point,
                           end=end_point,
                           prior=prior,
                           alpha=0.1,
                           gamma=0.9,
                           epsilon=0.1)

Precomputing duration of time between all pairs of parking structures.
http://routing.openstreetmap.de/routed-foot/route/v1/driving/-122.1765937,37.4262091;-122.17683561120828,37.428106
Roble Field Garage -> Via Ortega Garage : 1.058816511572417 mins
http://routing.openstreetmap.de/routed-foot/route/v1/driving/-122.1765937,37.4262091;-122.18208002336104,37.4316971
Roble Field Garage -> Stock Farm Garage : 3.6439692197566216 mins
http://routing.openstreetmap.de/routed-foot/route/v1/driving/-122.1765937,37.4262091;-122.1706597,37.4236765
Roble Field Garage -> Tressider Student Union : 2.772682534001432 mins
http://routing.openstreetmap.de/routed-foot/route/v1/driving/-122.1765937,37.4262091;-122.16409634597889,37.42308
Roble Field Garage -> Wilbur Field Garage : 5.360817823908376 mins
http://routing.openstreetmap.de/routed-foot/route/v1/driving/-122.17683561120828,37.428106;-122.18208002336104,37.4316971
Via Ortega Garage -> Stock Farm Garage : 3.5134812693867814 mins
http://routing.open

In [207]:
pp.pprint(q_learn.duration_matrix)

array([[0.        , 1.05881651, 3.64396922, 2.77268253, 5.36081782],
       [1.05881651, 0.        , 3.51348127, 3.76289967, 5.33173765],
       [3.64396922, 3.51348127, 0.        , 6.41963434, 8.3590581 ],
       [2.77268253, 3.76289967, 6.41963434, 0.        , 2.55756383],
       [5.36081782, 5.33173765, 8.3590581 , 2.55756383, 0.        ]])


In [208]:
q_learn.train(episodes=10000)

In [209]:
print(q_learn.start_duration)
print(q_learn.end_duration)

[8.316929133858268, 7.820702099737533, 9.04579754235266, 7.364739918873778, 7.000492125984253]
[5.853316630875685, 3.5169609480633097, 12.786576393859859, 11.041766086057425, 15.545464487393621]


In [210]:

policy = {}
for state, q_values in q_learn.table.items():
    mask = np.ones_like(q_values, dtype=bool)
    for visited in state:
        mask[visited] = False

    # mask off indices of parking structures we already visited
    masked_q_values = np.where(mask, q_values, -np.inf)
    best_action = np.argmax(masked_q_values)
    if best_action in state:
        policy[state] = None
    else:
        policy[state] = int(np.argmax(masked_q_values))


In [211]:
print(policy)

{(): 1, (0,): 2, (0, 1): 3, (0, 1, 2): 3, (0, 1, 2, 3): 4, (0, 1, 2, 3, 4): None, (1,): 2, (1, 0): 2, (1, 0, 2): 3, (1, 0, 2, 3): 4, (1, 0, 2, 3, 4): None, (2,): 0, (2, 0): 1, (2, 0, 1): 3, (2, 0, 1, 3): 4, (2, 0, 1, 3, 4): None, (3,): 4, (3, 0): 1, (3, 0, 1): 2, (3, 0, 1, 2): 4, (3, 0, 1, 2, 4): None, (4,): 3, (4, 0): 1, (4, 0, 1): 3, (4, 0, 1, 2): 3, (4, 0, 1, 2, 3): None, (4, 1): 0, (4, 1, 0): 3, (4, 1, 0, 2): 3, (4, 1, 0, 2, 3): None, (3, 1): 0, (3, 1, 0): 2, (3, 1, 0, 2): 4, (3, 1, 0, 2, 4): None, (1, 2): 0, (1, 2, 0): 3, (1, 2, 0, 3): 4, (1, 2, 0, 3, 4): None, (0, 2): 1, (0, 2, 1): 3, (0, 2, 1, 3): 4, (0, 2, 1, 3, 4): None, (2, 1): 0, (2, 1, 0): 3, (2, 1, 0, 3): 4, (2, 1, 0, 3, 4): None, (4, 2): 0, (4, 2, 0): 1, (4, 2, 0, 1): 3, (4, 2, 0, 1, 3): None, (3, 2): 0, (3, 2, 4): 0, (3, 2, 4, 0): 1, (3, 2, 4, 0, 1): None, (1, 3): 2, (1, 3, 0): 2, (1, 3, 0, 2): 4, (1, 3, 0, 2, 4): None, (0, 3): 2, (0, 3, 1): 2, (0, 3, 1, 2): 4, (0, 3, 1, 2, 4): None, (2, 3): 0, (2, 3, 0): 1, (2, 3, 0, 1)

In [212]:
def prune_unreachable_nodes(policy):
    """
    Prunes nodes from the policy dictionary that are not reachable
    when an agent always takes the optimal action.

    Args:
        policy (dict): A dictionary where keys are tuples representing states
                       and values are integers representing optimal actions.

    Returns:
        dict: A pruned policy dictionary containing only reachable nodes.
    """
    reachable = set()  # Set to store reachable nodes
    stack = [()]       # Start from the root node (empty sequence)

    while stack:
        current = stack.pop()
        if current in reachable:
            continue  # Skip if already visited

        reachable.add(current)
        optimal_action = policy.get(current)

        if optimal_action is not None:
            # Generate the next state by appending the optimal action
            next_state = current + (optimal_action,)
            stack.append(next_state)

    # Create a new policy dictionary with only reachable nodes
    pruned_policy = {state: action for state, action in policy.items() if state in reachable}
    return pruned_policy


pruned_policy = prune_unreachable_nodes(policy)
print(pruned_policy)

{(): 1, (1,): 2, (1, 2): 0, (1, 2, 0): 3, (1, 2, 0, 3): 4, (1, 2, 0, 3, 4): None}


In [213]:
names = list(parking_probability_map.keys())
print(names)
segments = list()
for state, action in pruned_policy.items():
    print(state, action)
    text_state = tuple([names[visited] for visited in state])
    name_state = ' -> '.join(text_state)
    print("Visited:", name_state)
    optimal_action_name = names[action] if action is not None else None
    if len(state) == 0:
        segments.append([start_point, parking_structures[action].location])
    elif action is None:
        segments.append([parking_structures[state[-1]].location, end_point])
    else:
        segments.append([parking_structures[state[-1]].location, parking_structures[action].location])
    print("Optimal Action:", optimal_action_name)
    print()

['Roble Field Garage', 'Via Ortega Garage', 'Stock Farm Garage', 'Tressider Student Union', 'Wilbur Field Garage']
() 1
Visited: 
Optimal Action: Via Ortega Garage

(1,) 2
Visited: Via Ortega Garage
Optimal Action: Stock Farm Garage

(1, 2) 0
Visited: Via Ortega Garage -> Stock Farm Garage
Optimal Action: Roble Field Garage

(1, 2, 0) 3
Visited: Via Ortega Garage -> Stock Farm Garage -> Roble Field Garage
Optimal Action: Tressider Student Union

(1, 2, 0, 3) 4
Visited: Via Ortega Garage -> Stock Farm Garage -> Roble Field Garage -> Tressider Student Union
Optimal Action: Wilbur Field Garage

(1, 2, 0, 3, 4) None
Visited: Via Ortega Garage -> Stock Farm Garage -> Roble Field Garage -> Tressider Student Union -> Wilbur Field Garage
Optimal Action: None



In [214]:
print(segments)

[[{'name': 'Galvez Street & El Camino Real', 'lat': 37.4369847, 'lon': -122.1611878}, {'name': 'Via Ortega Garage', 'lat': 37.428106, 'lon': -122.17683561120828}], [{'name': 'Via Ortega Garage', 'lat': 37.428106, 'lon': -122.17683561120828}, {'name': 'Stock Farm Garage', 'lat': 37.4316971, 'lon': -122.18208002336104}], [{'name': 'Stock Farm Garage', 'lat': 37.4316971, 'lon': -122.18208002336104}, {'name': 'Roble Field Garage', 'lat': 37.4262091, 'lon': -122.1765937}], [{'name': 'Roble Field Garage', 'lat': 37.4262091, 'lon': -122.1765937}, {'name': 'Tressider Student Union', 'lat': 37.4236765, 'lon': -122.1706597}], [{'name': 'Tressider Student Union', 'lat': 37.4236765, 'lon': -122.1706597}, {'name': 'Wilbur Field Garage', 'lat': 37.42308, 'lon': -122.16409634597889}], [{'name': 'Wilbur Field Garage', 'lat': 37.42308, 'lon': -122.16409634597889}, {'name': 'NVIDIA Auditorium', 'lat': 37.4281301, 'lon': -122.1742004}]]


In [215]:
!pip install folium polyline

In [218]:
import folium
import polyline

coords = [parking_structures[i].location.lat for i in range(len(parking_structures))], [parking_structures[i].location.lon for i in range(len(parking_structures))]
coordinates = list(zip(coords[0], coords[1]))
# Create a map centered at the first coordinate
map_center = parking_structures[0].location.lat, parking_structures[0].location.lon

mymap = folium.Map(location=map_center, zoom_start=15)

# Add markers for each coordinate
for idx, (lat, lon) in enumerate(coordinates):
    folium.Marker(location=[lat, lon], popup=f"Point {idx+1}").add_to(mymap)

folium.Marker(location=[start_point.lat, start_point.lon], popup="Start", icon=folium.Icon(color="green", icon="info-sign")).add_to(mymap)
folium.Marker(location=[end_point.lat, end_point.lon], popup="End", icon=folium.Icon(color="red", icon="info-sign")).add_to(mymap)

routes = [([seg[0].lat, seg[0].lon], [seg[1].lat, seg[1].lon]) for seg in segments]

map_stages = []
for i, route in enumerate(routes[:-1]):
    plot_routes = Location.get_routes(profile="driving", lat1=route[0][0], lon1=route[0][1], lat2=route[1][0], lon2=route[1][1], overview="simplified")
    folium.PolyLine(polyline.decode(plot_routes[0]["geometry"]), color="blue", weight=2.5, opacity=0.8).add_to(mymap)
    map_stages.append(mymap)
    mymap.save(f"map{i}.html")

            
plot_routes = Location.get_routes(profile="foot", lat1=routes[-1][0][0], lon1=routes[-1][0][1], lat2=routes[-1][1][0], lon2=routes[-1][1][1], overview="simplified")

folium.PolyLine(polyline.decode(plot_routes[0]["geometry"]), color="red", weight=2.5, opacity=0.8).add_to(mymap)
mymap.save(f"map{i}.html")
# Save the map to an HTML file


# Display the map directly in a Jupyter Notebook (if you're using one)
mymap


http://routing.openstreetmap.de/routed-foot/route/v1/driving/-122.1611878,37.4369847;-122.17683561120828,37.428106
http://routing.openstreetmap.de/routed-foot/route/v1/driving/-122.17683561120828,37.428106;-122.18208002336104,37.4316971
http://routing.openstreetmap.de/routed-foot/route/v1/driving/-122.18208002336104,37.4316971;-122.1765937,37.4262091
http://routing.openstreetmap.de/routed-foot/route/v1/driving/-122.1765937,37.4262091;-122.1706597,37.4236765
http://routing.openstreetmap.de/routed-foot/route/v1/driving/-122.1706597,37.4236765;-122.16409634597889,37.42308
http://routing.openstreetmap.de/routed-foot/route/v1/foot/-122.16409634597889,37.42308;-122.1742004,37.4281301
